
# Trying out second form of PDE

Implemented in modesolver as leakymode2 vecmodesystem2.  Debugging here.

In [ ]:
import numpy as np
import ngsolve as ng

from fiberamp.fiber.microstruct.bragg import Bragg
from ngsolve.webgui import Draw
from ngsolve import CF

from pyeigfeast import NGvecs, SpectralProjNGGeneral

In [ ]:
n_air = 1.00027717
n_glass = 1.4388164768221814
ts = [15*2.7183333333333333e-6, 15*2/3*1e-6, 15*1e-6]
ns = [lambda x: n_air, lambda x: n_glass, lambda x: n_glass]
mats = ['air', 'glass', 'Outer']
maxhs = [.1, .015, .025]
scale = 15e-6
i=0
wls = np.linspace(1.4, 2, 301) * 1e-6

A = Bragg(ts=ts, scale=scale, maxhs=maxhs, ns=ns, mats=mats, wl=wls[i])


In [ ]:
p = 3
alpha = .2

A.mesh.SetPML(ng.pml.Radial(rad=A.R, alpha=alpha*1j, origin=(0, 0)),'Outer')

k2 = A.k ** 2 * A.L**2 * A.index * A.index

dr = ng.ds(definedon=~A.mesh.Materials('Outer'), skeleton=1)
n = ng.specialcf.normal(2)

X = ng.HCurl(A.mesh, order=p+1-max(1-p, 0), type1=True,
             dirichlet='OuterCircle', dgjumps=True, complex=True)
Y = ng.H1(A.mesh, order=p+1, dgjumps=True, complex=True)
XY = ng.FESpace([X,Y])

(E, phi), (v, psi) = XY.TnT()

AA = ng.BilinearForm(XY)
AA += (ng.curl(E) * ng.curl(v) - k2 * E * v) * ng.dx
AA += ng.grad(phi) * v * ng.dx
AA += ng.grad(phi) * ng.grad(psi) * ng.dx
AA += -k2 * phi * psi * ng.dx
AA += -psi * (ng.grad(phi.Other()) - ng.grad(phi)) * n * dr

B= ng.BilinearForm(XY)
B += -E * v * ng.dx
B += -E * ng.grad(psi) * ng.dx
B += psi * (E.Other() - E) * n * dr

MM = ng.BilinearForm(XY)
MM += (E * v + phi * psi) * ng.dx

with ng.TaskManager():
    try:
        AA.Assemble()
        B.Assemble()
        MM.Assemble()

    except Exception:
        print('*** Trying again with larger heap')
        ng.SetHeapSize(int(1e9))
        AA.Assemble()
        B.Assemble()
        MM.Assemble()


In [ ]:
exact_scaled_betas = np.load('/home/pv/local/convergence/bragg_fiber/wavelength/N0/exact_scaled_betas.npy').conjugate()
centers = exact_scaled_betas **2

In [ ]:
ctr = centers[i]
rad = .01
npts = 2

P = SpectralProjNGGeneral(XY, AA.mat, B.mat, radius=rad, center=ctr,
                          npts=npts)

In [ ]:
nspan = 2

E_phi = NGvecs(XY, nspan, M=MM.mat)
E_phi_l = E_phi.create()
E_phi.setrandom(seed=1)
E_phi_l.setrandom(seed=1)

In [ ]:
beta_sqrs, E_phi, history, E_phi_l = P.feast(E_phi, Yl=E_phi_l, niterations=5, nrestarts=0,
                                             hermitian=False)

betas = np.sqrt(beta_sqrs, dtype=complex)/A.L
Zsqrs = A.sqrZfrom(betas)

In [ ]:
for e in E_phi:
    Draw(-1e3*e.components[0].real[0],A.mesh)

In [ ]:
for e in E_phi:
    Draw(1e2*e.components[1],A.mesh)

# Automated version hopefully

In [ ]:
alpha=1
nspan = 4
p = 1
center = centers[i]
radius = .1
npts = 2

beta, _, E_phis, _ = A.leakyvecmodes2(ctr=center,
                                       rad=radius,
                                       alpha=None,
                                       d=None,
                                       nspan=nspan,
                                       npts=npts,
                                       p=p,
                                       niterations=4,
                                       nrestarts=0,
                                       stop_tol=1e-9,)